In [2]:
import os
os.environ['GROQ_API_KEY'] = "YOUR API KEY FROM GROQ"

In [3]:
!pip install langchain_groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 5.8 MB/s eta 0:00:00


In [4]:
!pip install -q langchain_community langchain_groq faiss-cpu tiktoken python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [5]:

!pip install youtube-transcript-api==0.6.2


In [6]:
!pip install -q yt_dlp beautifulsoup4 requests


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.9/175.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 71.0 MB/s eta 0:00:00


In [7]:

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate

Step1 indexing (document injestion)

In [10]:
import yt_dlp
import json

def get_clean_transcripts(url, lang="en"):
    ydl_opts = {
        "skip_download": True,
        "writeautomaticsub": True,
        "subtitlesformat": "json3",
        "subtitleslangs": [lang],
        "cookiefile": "demo_cookies.txt"
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=False)
        subs = info.get("automatic_captions") or info.get("subtitles")
        if not subs or lang not in subs:
            print("❌ No subtitles available")
            return []

        sub_url = subs[lang][0]["url"]

        import requests
        response = requests.get(sub_url)
        data = response.text.strip()
        if not data:
            print("❌ Subtitle file empty")
            return []

        try:
            json_data = json.loads(data)
        except json.JSONDecodeError:
            # YouTube JSON sometimes comes as multiple lines; fix it
            json_data = json.loads("{" + data.split("{", 1)[1])

        results = []
        for event in json_data.get("events", []):
            if "segs" not in event:
                continue
            start = event.get("tStartMs", 0) / 1000.0
            duration = event.get("dDurationMs", 0) / 1000.0
            text = "".join(seg.get("utf8", "") for seg in event["segs"]).replace("\n", " ").strip()
            if text:
                results.append({
                    "text": text,
                    "start": round(start, 3),
                    "duration": round(duration, 3)
                })
        return results

In [11]:
url = "https://www.youtube.com/watch?v=X4PUoZSPlw0"
transcript = get_clean_transcripts(url)
print( transcript if transcript else " No transcript found.")

[youtube] Extracting URL: https://www.youtube.com/watch?v=X4PUoZSPlw0
[youtube] X4PUoZSPlw0: Downloading webpage
[youtube] X4PUoZSPlw0: Downloading tv client config
[youtube] X4PUoZSPlw0: Downloading tv player API JSON
[youtube] X4PUoZSPlw0: Downloading web safari player API JSON
[youtube] X4PUoZSPlw0: Downloading player 0004de42-main
[youtube] X4PUoZSPlw0: Downloading m3u8 information
[info] X4PUoZSPlw0: Downloading subtitles: en
[{'text': 'This might just be the biggest change to', 'start': 0.16, 'duration': 4.56}, {'text': 'NN since well, it even existed. And I', 'start': 1.68, 'duration': 5.84}, {'text': 'got early access. What if you could', 'start': 4.72, 'duration': 5.68}, {'text': 'build a complete AI agent or automation', 'start': 7.52, 'duration': 5.199}, {'text': 'workflow simply just by describing what', 'start': 10.4, 'duration': 4.319}, {'text': 'you want in plain English? No more', 'start': 12.719, 'duration': 5.121}, {'text': "dragging around boxes for 8 hours. It's", '

In [12]:
# Combine list of dicts into one single text string
combined_text = " ".join(chunk["text"] for chunk in transcript)


In [14]:
print(combined_text)

This might just be the biggest change to NN since well, it even existed. And I got early access. What if you could build a complete AI agent or automation workflow simply just by describing what you want in plain English? No more dragging around boxes for 8 hours. It's literally just the chat GPT for NAND. I just built a workflow that would have taken me hours within the last few minutes. And let me show you how. This is NadN's new AI workflow builder, also called text to workflow, and they made it easier than ever somehow to actually go ahead and build AI agents without writing a single line of code. Hey, I'm Josh and I've been covering NAN automations on this channel for the past year. And NAN just invited me to their limited beta for this new tool. And honestly, I'm not sure if I'm excited or a bit concerned for my own job security because here's the thing. remember spending hours connecting nodes, debugging web hooks, and wrestling with JSON? Yeah, that might be over. In this video

step 1 b , indexing: text splitting

In [15]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

In [16]:
chunks = splitter.create_documents([combined_text])

In [17]:
len(chunks)

27

In [18]:
chunks[5]

Document(metadata={}, page_content="that it's updating save report node parameters. So we can see all the different kind of tools that it has access to and that it's using throughout this process. All right. So now it is officially done. We can see that it's giving us a setup guide. So how to set this up? Configure our open AI credentials, our SER API, update the safe report node URL to your actual report storage endpoint, modify the research topic in the N workflow configuration as needed. And we can see this workflow runs automatically at 9:00 a.m. You can also trigger it manually to test and the system will research the configured topic using the web search fact check findings for accuracy, compile a comprehensive structured report and save the final report to your specified endpoint. Now let's go ahead and take a look at what it actually built us here. So we can literally see just that. Right here we have a research schedule. So 9:00 a.m. node right here. Then we have our workflow 

embedding generation and storing in vector store

In [19]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {"device": "cpu"}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs
)

/tmp/ipython-input-3044703104.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [24]:
vector_store = FAISS.from_documents(chunks, embeddings)

In [23]:
vector_store.index_to_docstore_id

{0: '1fc276dc-9ff6-4fbc-93b9-66dfa8080236',
 1: '9ad937c7-920f-4f5a-95fb-d9c60ba2f3a9',
 2: '55024ad4-db64-44ef-9a40-d7579e1aeb3a',
 3: 'd5058ffb-a6ab-43f3-ae52-0238429534e6',
 4: 'd25ead09-ab6a-4564-9bfa-04acdfe1ff62',
 5: '2e29b61e-dea1-4d16-a112-e2bd6e1ff8f9',
 6: 'b2211b98-a1e8-4b59-8b9c-38afa0fd55a4',
 7: '1c3c9368-8cb5-4df4-ba2a-6c138563c566',
 8: 'cb6aa7ef-b2e0-41ec-a289-1c68f49ee6f5',
 9: '39f1e7a5-ba28-46fc-b9f3-b64aa90a88a9',
 10: '409d9c36-fab8-4766-b514-7b539eb9fea6',
 11: '981545be-5860-4cdd-b325-b07d36624339',
 12: 'f011540e-65c0-4f26-9fa5-77fd9a78ef97',
 13: 'fc0a70e6-3de2-4da3-97ad-243033a24400',
 14: '64d11657-2e7f-46d5-8129-5d03154a8b8c',
 15: 'db1c2e90-81c1-4e1f-81d9-6045c5eb66b5',
 16: '429c4f10-d7dc-47fa-9d90-25f76e06647b',
 17: '65c304fe-9bd5-4b84-9c2a-05cff72577ef',
 18: '79c9ef55-0b00-4347-a539-6953e82dd397',
 19: '64a18931-783d-4928-88ec-8e00d1128e0d',
 20: '61f9e69f-33a3-4b5f-966a-bb62bf3c70cf',
 21: '8d46a629-233d-4f06-8b0a-f8cab45d5e37',
 22: '47499e55-01b6-

In [28]:
vector_store.get_by_ids(['4c0def56-4792-4aa4-9c08-5b044c7603af'])

[]

2. creating retriever


In [29]:
retriever = vector_store.as_retriever(search_type = "similarity", search_kwargs = {"k": 4})

In [30]:
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7b421c26ee10>, search_kwargs={'k': 4})

In [31]:
retriever.invoke("what is agent")

[Document(id='a89f5bb8-3793-4ab8-948b-718943b8bfd7', metadata={}, page_content="another issue here for the fact check agent. So, I'm going to paste this back in here. All right. So, I just actually asked Claude about this and it's basically saying that there is a circular tool dependency which makes sense. So, I just pasted that into NAI here and then it's basically saying you're absolutely right about the circular dependency here and now it actually went ahead and executed on that should make things a lot simpler which looks like it did a good job on. So, we can go ahead and test this now. So, I encountered another issue here where the fact check agent wasn't running. So, I actually just asked Claude about it as well. It gave me an answer and I simply just paste that into NAI here to execute on. And let's see what it actually does. Okay, so now we got a code node right here after the research agent which formats the data for the fact check agent before it does the merge node. So, let'

3. Augumentation

In [42]:
# model = HuggingFaceEndpoint(
#     repo_id="mistralai/Mistral-7B-Instruct-v0.3",
#     task="text-generation"
# )


# llm = ChatHuggingFace(llm = model)
llm = ChatGroq(
    model="meta-llama/llama-4-maverick-17b-128e-instruct",
    temperature=0.2,
    max_tokens=512
)

In [39]:
prompt = PromptTemplate(
    template = """
      your are a helpful assitant. Answer only from the provided transcript Context. If the context is insufficient, just say you dont know.
      {context}
      Question : {question}
    """,
    input_variables=['context','question']
)

In [44]:
question = "is the topic neuclear reaction atom bomb discussed in video? if yes then what was discussed"
retrieved_docs = retriever.invoke(question)

In [46]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)

In [47]:
final_prompt = prompt.invoke({'context': context_text, 'question': question})

4 generation

In [48]:
answer = llm.invoke(final_prompt)
print(answer.content)

No, the topic "nuclear reaction atom bomb" is not discussed in the video. The transcript appears to be discussing a multi-agent AI workflow, N8N AI, and automations, but does not mention nuclear reactions or atom bombs.


Chain & runnable

In [50]:
"""
         question -> question ->
                                     prompt > llm > parser -> answer
question > retriever > context >
"""
# ie first make parallel chain and then final sequential chain

'\n         question -> question ->\n                                     prompt > llm > parser -> answer\nquestion > retriever > context > \n'

In [51]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [53]:
#function to return string from list of documents of transcript text
def format_docs(retrieved_docs):
  context_text = "\n \n".join(doc.page_content for doc in retrieved_docs)
  return context_text

In [54]:
parallel_chain = RunnableParallel({
    'context':retriever | RunnableLambda(format_docs),
    'question':RunnablePassthrough()
})


In [55]:
parallel_chain.invoke("what is agent")

{'context': "another issue here for the fact check agent. So, I'm going to paste this back in here. All right. So, I just actually asked Claude about this and it's basically saying that there is a circular tool dependency which makes sense. So, I just pasted that into NAI here and then it's basically saying you're absolutely right about the circular dependency here and now it actually went ahead and executed on that should make things a lot simpler which looks like it did a good job on. So, we can go ahead and test this now. So, I encountered another issue here where the fact check agent wasn't running. So, I actually just asked Claude about it as well. It gave me an answer and I simply just paste that into NAI here to execute on. And let's see what it actually does. Okay, so now we got a code node right here after the research agent which formats the data for the fact check agent before it does the merge node. So, let's go ahead and test this and see if it works. All right. So, after 

In [56]:
parser = StrOutputParser()

In [57]:
main_chain = parallel_chain | prompt | llm | parser

In [58]:
main_chain.invoke("can you summarize the video")

'The video is about the release of an AI agent workflow builder in NAND, which allows users to create automation workflows without requiring technical knowledge. The speaker highlights that this tool can create a working prototype in minutes, and that the market for automation is about to explode, making expertise in NAND more valuable. The video demonstrates how to use the workflow builder, specifically a "multi-agent research workflow" that collaborates to research a topic and compile reports. The speaker also mentions an 8-page document outlining the NAND AI workflow builder and how to capitalize on the opportunity, which will be available in their free Stride AI Academy.'